### Experiment: Pre-processing and frequency sampling

**Question**: Is it possible to train a model on unpreprocessed EEG data and still attain similar performance levels?

**Hypothesis**: The model will perform worse, but if still similar then the added value of not having to (manually) preprocess EEG data is very valuable and opens up a multitude of applications.

**Result**:

#### Part 1: Preparing data
To use hmp.utils.read_mne_data() and epoch the information, the files should be in .fif format, this replicates automated preprocessing as done in https://github.com/GWeindel/hsmm_mvpy/blob/main/tutorials/sample_data/eeg/0022.ipynb excepting resampling to 100Hz

In [1]:
import mne
from pathlib import Path
import hsmm_mvpy as hmp
import pandas as pd
import numpy as np
import xarray as xr
from shared.data import add_stage_dimension

2023-10-16 11:33:18.394045: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 11:33:19.015391: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Set up paths and file locations
data_path = Path("/mnt/d/thesis/sat1/")
behavioral_data_path = data_path / "ExperimentData/ExperimentData"
output_path = Path("data/sat1/unpreprocessed")

subj_ids = [
    subj_id.name.split("-")[1][:4] for subj_id in (data_path / "eeg4").glob("*.vhdr")
]
subj_files = [
    str(output_path / f"unprocessed_{subj_id}_epo.fif") for subj_id in subj_ids
]
behavioral_files = [
    str(behavioral_data_path / f"{subj_id}-cnv-sat3_ET.csv") for subj_id in subj_ids
]

In [3]:
# Replacing preprocessing done in https://github.com/GWeindel/hsmm_mvpy/blob/main/tutorials/sample_data/eeg/0022.ipynb
# with only the necessary (non-manual) parts, like adding metadata for processing in HMP package, more info in link above
for subject_id in subj_ids:
    print(f"Processing subject: {subject_id}")
    subject_id_short = subject_id.replace("0", "")
    raw = mne.io.read_raw_brainvision(
        data_path / "eeg4" / f"MD3-{subject_id}.vhdr", preload=False
    )
    raw.set_channel_types(
        {"EOGh": "eog", "EOGv": "eog", "A1": "misc", "A2": "misc"}
    )  # Declare type to avoid confusion with EEG channels
    raw.rename_channels({"FP1": "Fp1", "FP2": "Fp2"})  # Naming convention
    raw.set_montage("standard_1020")  # Standard 10-20 electrode montage
    raw.rename_channels({"Fp1": "FP1", "Fp2": "FP2"})

    behavioral_path = behavioral_data_path / f"{subject_id}-cnv-sat3_ET.csv"
    behavior = pd.read_csv(behavioral_path, sep=";")[
        [
            "stim",
            "resp",
            "RT",
            "cue",
            "movement",
        ]
    ]
    behavior["movement"] = behavior.apply(
        lambda row: "stim_left"
        if row["movement"] == -1
        else ("stim_right" if row["movement"] == 1 else np.nan),
        axis=1,
    )
    behavior["resp"] = behavior.apply(
        lambda row: "resp_left"
        if row["resp"] == 1
        else ("resp_right" if row["resp"] == 2 else np.nan),
        axis=1,
    )
    # Merging together the exeperimental conditions info to have the format condition/stimulus/response
    behavior["trigger"] = (
        behavior["cue"] + "/" + behavior["movement"] + "/" + behavior["resp"]
    )
    # Filtering out < 300 and > 3000 Reaction times
    behavior["RT"] = behavior.apply(
        lambda row: 0
        if row["RT"] < 300
        else (0 if row["RT"] > 3000 else float(row["RT"]) / 1000),
        axis=1,
    )
    epochs = mne.io.read_epochs_fieldtrip(
        data_path / "eeg1" / f"data{subject_id_short}.mat", info=raw.info
    )
    epochs.rename_channels({"FP1": "Fp1", "FP2": "Fp2"})  # Naming convention
    epochs.set_montage("easycap-M1")
    epochs.filter(1, 35)  # Bandwidth filter from van Maanen, Portoles & Borst (2021)
    epochs.crop(tmin=-0.250)
    epochs.set_eeg_reference("average")
    epochs.metadata = behavior
    epochs.save(
        output_path / f"unprocessed_{subject_id}_epo.fif", overwrite=True, verbose=False
    )  # Saving EEG mne format

Processing subject: 0001
Extracting parameters from /mnt/d/thesis/sat1/eeg4/MD3-0001.vhdr...
Setting channel info structure...


/tmp/ipykernel_11718/4173686901.py:9: RuntimeWarning: The unit for channel(s) A1, A2 has changed from V to NA.
  raw.set_channel_types(
/tmp/ipykernel_11718/4173686901.py:13: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['A1', 'A2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw.set_montage("standard_1020")  # Standard 10-20 electrode montage


Not setting metadata
200 matching events found
No baseline correction applied
Setting up band-pass filter from 1 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Adding metadata with 6 columns


...

/tmp/ipykernel_11718/4173686901.py:9: RuntimeWarning: The unit for channel(s) A1, A2 has changed from V to NA.
  raw.set_channel_types(
/tmp/ipykernel_11718/4173686901.py:13: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['A1', 'A2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw.set_montage("standard_1020")  # Standard 10-20 electrode montage


Not setting metadata
200 matching events found
No baseline correction applied
Setting up band-pass filter from 1 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Adding metadata with 6 columns


[Parallel(n_jobs=1)]: Done 6000 out of 6000 | elapsed:    1.4s finished


In [4]:
output_path_data = Path("data/sat1/data_unprocessed_500hz.nc")
# Run if data_unprocessed.nc does not exist or should be rewritten
data = hmp.utils.read_mne_data(
    subj_files,
    epoched=True,
    lower_limit_RT=0.2,
    upper_limit_RT=2,
    verbose=False,
    subj_idx=subj_ids,
    rt_col="RT",
)
data.to_netcdf(output_path_data)

Processing participant data/sat1/unpreprocessed/unprocessed_0001_epo.fif's epoched eeg
198 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0001_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0002_epo.fif's epoched eeg
200 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0002_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0003_epo.fif's epoched eeg
191 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0003_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0004_epo.fif's epoched eeg
200 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0004_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0005_epo.fif's epoched eeg
190 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0005_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0006_epo.fif's epoched eeg
200 trials were retaine

In [5]:
output_path_data = Path("data/sat1/data_unprocessed_100hz.nc")
# Run if data_unprocessed.nc does not exist or should be rewritten
data = hmp.utils.read_mne_data(
    subj_files,
    epoched=True,
    lower_limit_RT=0.2,
    upper_limit_RT=2,
    sfreq=100,
    verbose=False,
    subj_idx=subj_ids,
    rt_col="RT",
)
data.to_netcdf(output_path_data)

Processing participant data/sat1/unpreprocessed/unprocessed_0001_epo.fif's epoched eeg
198 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0001_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0002_epo.fif's epoched eeg
200 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0002_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0003_epo.fif's epoched eeg
191 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0003_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0004_epo.fif's epoched eeg
200 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0004_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0005_epo.fif's epoched eeg
190 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0005_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0006_epo.fif's epoched eeg
200 trials were retaine

#### Use information from stage_data to split unprocessed data

##### 500Hz

In [6]:
data_path = Path("data/sat1/stage_data.nc")
merge_dataset = xr.load_dataset(Path("data/sat1/data_unprocessed_500hz.nc"))
output_data = add_stage_dimension(data_path, merge_dataset)

Finding stage changes
Combining segments


In [7]:
output_path = Path("data/sat1/split_stage_data_unprocessed_500hz.nc")
output_data.to_netcdf(output_path)

##### 100Hz

In [8]:
data_path = Path("data/sat1/stage_data.nc")
merge_dataset = xr.load_dataset(Path("data/sat1/data_unprocessed_100hz.nc"))
output_data = add_stage_dimension(data_path, merge_dataset)

Finding stage changes
Combining segments


In [9]:
output_path = Path("data/sat1/split_stage_data_unprocessed_100hz.nc")
output_data.to_netcdf(output_path)

#### Part 2: Experiment

In [8]:
import tensorflow as tf
import gc
from pathlib import Path
from shared.data import add_stage_dimension
from shared.training import split_data_on_participants, train_and_evaluate, k_fold_cross_validate, compile_kwargs
from shared.normalization import *
from shared.models import SAT1Base, SAT1Topological, SAT1Deep
from shared.utilities import print_results
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


In [9]:
logs_path = Path("logs/exp_preprocessing/")

##### 2a: Processed 100Hz (control)

In [10]:
data_path = Path("data/sat1/split_stage_data.nc")
data = xr.load_dataset(data_path)

In [11]:
tf.keras.backend.clear_session()
model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
model.compile(**compile_kwargs)
train_kwargs = {
    "logs_path": logs_path,
    "additional_info": {"preprocessing": "default_100hz"},
    "additional_name": f"preprocessing-default_100hz",
}
results = k_fold_cross_validate(
    data, model, 5, normalization_fn=norm_min1_to_1, train_kwargs=train_kwargs
)
print_results(results)
del model
gc.collect()

2023-10-13 14:32:46.462449: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-13 14:32:46.492770: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-13 14:32:46.492844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-13 14:32:46.494392: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-13 14:32:46.494454: I tensorflow/compile

Fold 1: test fold: ['0009' '0017' '0001' '0024' '0012']
Epoch 1/10


2023-10-13 14:32:49.952084: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-13 14:32:50.668131: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-13 14:32:50.999124: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e937b27790 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-13 14:32:50.999153: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-10-13 14:32:51.005412: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-13 14:32:51.113625: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

999/999 [==============================] - 11s 7ms/step - loss: 0.7810 - accuracy: 0.7108
Epoch 2/10
999/999 [==============================] - 7s 7ms/step - loss: 0.5857 - accuracy: 0.7909
Epoch 3/10
999/999 [==============================] - 7s 7ms/step - loss: 0.5152 - accuracy: 0.8123
Epoch 4/10
999/999 [==============================] - 7s 7ms/step - loss: 0.4757 - accuracy: 0.8256
Epoch 5/10
999/999 [==============================] - 7s 7ms/step - loss: 0.4537 - accuracy: 0.8356
Epoch 6/10
999/999 [==============================] - 8s 8ms/step - loss: 0.4187 - accuracy: 0.8469
Epoch 7/10
999/999 [==============================] - 8s 8ms/step - loss: 0.3975 - accuracy: 0.8556
Epoch 8/10
999/999 [==============================] - 8s 7ms/step - loss: 0.3726 - accuracy: 0.8654
Epoch 9/10
999/999 [==============================] - 8s 8ms/step - loss: 0.3568 - accuracy: 0.8664
Epoch 10/10
254/254 [==============================] - 3s 10ms/step
Fold 1: accuracy: 0.8307086614173228
Fold 

1793

##### 2b: Unprocessed 100Hz

In [12]:
data_path = Path("data/sat1/split_stage_data_unprocessed_100hz.nc")
data = xr.load_dataset(data_path)

In [13]:
tf.keras.backend.clear_session()
model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
model.compile(**compile_kwargs)
train_kwargs = {
    "logs_path": logs_path,
    "additional_info": {"preprocessing": "unprocessed_100hz"},
    "additional_name": f"preprocessing-unprocessed_100hz",
}
results = k_fold_cross_validate(
    data, model, 5, normalization_fn=norm_min1_to_1, train_kwargs=train_kwargs
)
print_results(results)
del model
gc.collect()

Fold 1: test fold: ['0009' '0017' '0001' '0024' '0012']
Epoch 1/10
999/999 [==============================] - 9s 7ms/step - loss: 0.9692 - accuracy: 0.6186
Epoch 2/10
999/999 [==============================] - 7s 7ms/step - loss: 0.6561 - accuracy: 0.7584
Epoch 3/10
999/999 [==============================] - 8s 7ms/step - loss: 0.5641 - accuracy: 0.7929
Epoch 4/10
999/999 [==============================] - 8s 7ms/step - loss: 0.5167 - accuracy: 0.8125
Epoch 5/10
999/999 [==============================] - 8s 7ms/step - loss: 0.4811 - accuracy: 0.8257
Epoch 6/10
999/999 [==============================] - 8s 7ms/step - loss: 0.4537 - accuracy: 0.8368
Epoch 7/10
999/999 [==============================] - 8s 7ms/step - loss: 0.4363 - accuracy: 0.8415
Epoch 8/10
999/999 [==============================] - 8s 7ms/step - loss: 0.4158 - accuracy: 0.8494
Epoch 9/10
999/999 [==============================] - 8s 7ms/step - loss: 0.3930 - accuracy: 0.8599
Epoch 10/10
251/251 [=======================

54357

##### 2c: Unprocessed 500Hz

In [14]:
data_path = Path("data/sat1/split_stage_data_unprocessed_500hz.nc")
data = xr.load_dataset(data_path)

In [15]:
tf.keras.backend.clear_session()
model = SAT1Deep(len(data.channels), len(data.samples), len(data.labels))
model.compile(**compile_kwargs)
train_kwargs = {
    "logs_path": logs_path,
    "additional_info": {"preprocessing": "unprocessed_500hz"},
    "additional_name": f"preprocessing-unprocessed_500hz",
}
results = k_fold_cross_validate(
    data, model, 5, normalization_fn=norm_min1_to_1, train_kwargs=train_kwargs
)
print_results(results)
del model
gc.collect()

Fold 1: test fold: ['0009' '0017' '0001' '0024' '0012']
Epoch 1/10


2023-10-13 14:58:39.461825: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


999/999 [==============================] - 49s 45ms/step - loss: 1.2078 - accuracy: 0.4944
Epoch 2/10
999/999 [==============================] - 45s 45ms/step - loss: 1.0380 - accuracy: 0.5850
Epoch 3/10
999/999 [==============================] - 44s 44ms/step - loss: 0.8440 - accuracy: 0.6839
Epoch 4/10
999/999 [==============================] - 44s 43ms/step - loss: 0.7151 - accuracy: 0.7388
Epoch 5/10
999/999 [==============================] - 44s 43ms/step - loss: 0.6575 - accuracy: 0.7577
Epoch 6/10
999/999 [==============================] - 44s 44ms/step - loss: 0.6195 - accuracy: 0.7760
Epoch 7/10
999/999 [==============================] - 43s 43ms/step - loss: 0.5738 - accuracy: 0.7956
Epoch 8/10
999/999 [==============================] - 44s 43ms/step - loss: 0.5430 - accuracy: 0.8089
Epoch 9/10
999/999 [==============================] - 44s 43ms/step - loss: 0.5072 - accuracy: 0.8171
Epoch 10/10
251/251 [==============================] - 3s 11ms/step
Fold 1: accuracy: 0.81225

KeyboardInterrupt: 

In [13]:
# View results in Tensorboard
! tensorboard --logdir logs/exp_preprocessing/

2023-09-25 13:59:04.226414: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.13.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


: 